# This Notebook is Designed for Sentiment Analysis

#### 1. Load library and Read Data

In [1]:
!pip install -qU jsonlines
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [2]:
# Load model directly
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType, replace_lora_weights_loftq, prepare_model_for_kbit_training
import torch
import jsonlines
from datasets import Dataset, DatasetDict
import warnings
from kaggle_secrets import UserSecretsClient
import os
from accelerate import Accelerator

In [3]:
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Your device:", device)
user_secrets = UserSecretsClient()
os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"]="Sentiment-Analysis-Tiki"

Your device: cuda


In [4]:
with jsonlines.open('/kaggle/input/reviews-tiki/reviews_main.jsonl') as reader:
    data = []
    for obj in reader:
        review = {}
        if obj["content"] == "":
            continue
        review["label"] = int(obj["rating"]) - 1
        review["text"] = obj["content"]
        data.append(review)

#### 2. Split, Transform Data

In [5]:
dataset = Dataset.from_list(data)
train_size = int(len(dataset) * 0.9)
val_size = len(dataset) - train_size

In [6]:
shuffled_dataset = dataset.shuffle(seed=42, writer_batch_size=10_000)
train_dataset = shuffled_dataset.take(train_size)
shuffled_dataset = shuffled_dataset.skip(train_size)
val_dataset = shuffled_dataset.take(val_size)

In [7]:
dataset_dict = DatasetDict()
dataset_dict['train'] = train_dataset
dataset_dict['validation'] = val_dataset
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 56448
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 6272
    })
})

In [8]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
dataset_encoded = dataset_dict.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/56448 [00:00<?, ? examples/s]

Map:   0%|          | 0/6272 [00:00<?, ? examples/s]

In [10]:
dataset_encoded = dataset_encoded.remove_columns(["text"])
dataset_encoded = dataset_encoded.rename_column("label", "labels")
dataset_encoded.set_format("torch")

#### 3. Prepare Model, Metrics and Trainer

In [11]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [12]:
device_index = Accelerator().process_index
device_map = {"": device_index}

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased",
                                                            num_labels=5,
                                                            device_map=device_map)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Specify the task type as sequence classification
    r=16,  # Rank of the low-rank matrices
    lora_alpha=64,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate  
    target_modules=["query", "key", "value"] # which modules
)

peft_model_lora = get_peft_model(model, lora_config)

In [14]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [15]:
batch_size = 32
logging_steps = len(dataset_encoded["train"]) // batch_size // 5
model_name = "sentiment-analysis-finetuned-lora"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  disable_tqdm=False,
                                  eval_strategy="steps",
                                  logging_strategy="steps",
                                  logging_steps=logging_steps,
                                  eval_steps=logging_steps,
                                  push_to_hub=False,
                                  report_to="wandb",
                                  log_level="error")

In [16]:
trainer = Trainer(model=peft_model_lora,
                  args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=dataset_encoded["train"],
                  eval_dataset=dataset_encoded["validation"],
                  data_collator=data_collator,
                  tokenizer=tokenizer)

In [17]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ngocnguyen14073. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240928_034640-irhj2hgx
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sentiment-analysis-finetuned-lora
wandb: ⭐️ View project at https://wandb.ai/ngocnguyen14073/Sentiment-Analysis-Tiki
wandb: 🚀 View run at https://wandb.ai/ngocnguyen14073/Sentiment-Analysis-Tiki/runs/irhj2hgx


Step,Training Loss,Validation Loss,Accuracy,F1
352,0.720400,0.448860,0.898119,0.849912
704,0.448300,0.430876,0.898119,0.849912
1056,0.430100,0.400303,0.898119,0.849912
1408,0.402900,0.400509,0.898119,0.849912
1760,0.416700,0.385375,0.898119,0.849912
2112,0.386200,0.383888,0.898119,0.849912
2464,0.389400,0.383661,0.900510,0.864089
2816,0.371100,0.384281,0.899075,0.866360
3168,0.385500,0.385102,0.898756,0.866995
3520,0.389500,0.377851,0.899713,0.866509


TrainOutput(global_step=3528, training_loss=0.4340678036483237, metrics={'train_runtime': 2220.2286, 'train_samples_per_second': 50.849, 'train_steps_per_second': 1.589, 'total_flos': 1.1831093922227712e+16, 'train_loss': 0.4340678036483237, 'epoch': 2.0})

In [18]:
trainer.save_model("./latest_model")

In [19]:
# from transformers import pipeline

# classifier = pipeline('text-classification', model='./latest_model')

# result = classifier("Sản phẩm này rất tốt!")
# print(result)